# Test lógico negación

In [26]:
import sys

sys.path.append("../src")

In [27]:
import nltk
import pandas as pd

from pathlib import Path
from tqdm.auto import tqdm

In [28]:
from config.config import PATHS
from logic.logic_tester import LogicTester

In [41]:
data_folder = PATHS['training_data_folder']
data_path = Path(data_folder, 'negacion_5_fol.csv')
data = pd.read_csv(data_path, names=['Sentence 1', 'Sentence 2', 'Relation'], sep=";")
print(f'Data shape: {data.shape}')
data.tail()

Data shape: (10650, 3)


,Sentence 1,Sentence 2,Relation
10645,exists x.(BARCO(x) & (CALLAR(x) | PASEAR(x))),all x.(BARCO(x) -> (CALLAR(x) | PASEAR(x))),0
10646,exists x.(BARCO(x) & (PASEAR(x) | CAMBIAR(x))),all x.(BARCO(x) -> (PASEAR(x) | CAMBIAR(x))),0
10647,exists x.(BARCO(x) & (PASEAR(x) | DORMIR(x))),all x.(BARCO(x) -> (PASEAR(x) | DORMIR(x))),0
10648,exists x.(BARCO(x) & (PASEAR(x) | TRANSCURRIR(...,all x.(BARCO(x) -> (PASEAR(x) | TRANSCURRIR(x))),0
10649,exists x.(BARCO(x) & (PASEAR(x) | CALLAR(x))),all x.(BARCO(x) -> (PASEAR(x) | CALLAR(x))),0


In [42]:
sentences1 = data['Sentence 1'].tolist()
sentences2 = data['Sentence 2'].tolist()

def translation_to_prover(sentence:str) -> str:
    '''
    Translate a sentence to prover format.
    '''
    sentence_nltk = lp.parse(f'{sentence}')
    sentence_pyprover = Parse2pyprover.parse(sentence_nltk)
    return sentence_pyprover

def check_implication(premisas:List[any], conclusion:any) -> bool:
    resultado = proves(premisas, conclusion)
    return resultado

In [43]:
tester = LogicTester()

In [44]:
tested = list()
pbar = tqdm(total=data.shape[0], desc="Testing negation relations")
for index, row in data.iterrows():
    sentences1 = row['Sentence 1']
    sentences2 = row['Sentence 2']
    # print('Sentences:', sentences1, sentences2)
    relation = row['Relation']
    result = tester.test_negacion(sentences1, sentences2)
    tested.append(int(result))
    pbar.update(1)

data['Tested'] = tested
data.head()

Testing negation relations:   0%|          | 0/10650 [00:00<?, ?it/s]

,Sentence 1,Sentence 2,Relation,Tested
0,all x.(AVIÓN(x) -> CAMBIAR(x)),exists x.(AVIÓN(x) & -CAMBIAR(x)),1,1
1,all x.(AVIÓN(x) -> DORMIR(x)),exists x.(AVIÓN(x) & -DORMIR(x)),1,1
2,all x.(AVIÓN(x) -> TRANSCURRIR(x)),exists x.(AVIÓN(x) & -TRANSCURRIR(x)),1,1
3,all x.(AVIÓN(x) -> CALLAR(x)),exists x.(AVIÓN(x) & -CALLAR(x)),1,1
4,all x.(AVIÓN(x) -> PASEAR(x)),exists x.(AVIÓN(x) & -PASEAR(x)),1,1


In [45]:
errors = pd.DataFrame(data[data['Tested'] != data['Relation']]).reset_index()

In [ ]:
len(errors)

In [ ]:
errors.head()

,index,Sentence 1,Sentence 2,Relation,Tested


In [37]:
# idx, sentence1, sentence2, relation, test = errors.loc[1,:].values.tolist()
# print(f"Sentence 1: {sentence1}")
# print(f"Sentence 2: {sentence2}")